In [147]:
import numpy as np
import pandas as pd
import sklearn.datasets
%matplotlib inline
import matplotlib.pyplot as plt
import random
from sklearn.preprocessing import Imputer
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import train_test_split


In [148]:
df = pd.read_csv('datasets/startup.csv', index_col=0)
colsToDrop=['ICBsector','Lat','Lon','accelaratorResult','Old value','roundDate','equityValuation','investorName', 'foundedDate','state','city','dealInvestmentAmount','accelerator','roundInvestorCount','linkedinURL','roundLeadInvestorType','roundValuation','accleratorDate','incubatorResult','website','source','founderName','description','incubator','incubatorDate']
#df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1)
df = df.drop(colsToDrop, axis=1)
df = df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1)
#print df.head(2)
print df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 111 entries, YNOS0309 to YNOS0541
Data columns (total 11 columns):
startupName              111 non-null object
ICBindustry              81 non-null object
businessModel            81 non-null object
groupClassification      111 non-null object
investmentStage          74 non-null object
investorType             74 non-null object
keyword                  111 non-null object
roundInvestmentAmount    111 non-null object
startupClassification    111 non-null object
stageClassification      84 non-null object
startupStatus            77 non-null object
dtypes: object(11)
memory usage: 10.4+ KB
None


In [149]:
#Pre-seed is 0, Seed is 1, Series A is 2, Series B is 3, Series C is 4, Others is 5 and blank is -1
def findStage(classification):
    round_name, = classification
    if(type(round_name)==float):
        return -1
    if(round_name.find("Pre-Seed")!=-1):
        return 0
    elif (round_name.find("Seed")!=-1):
        return 1
    elif(round_name.find("Series A")!=-1):
        return 2
    elif(round_name.find("Series B")!=-1):
        return 3
    elif(round_name.find("Series C")!=-1):
        return 4
    else:
        return 5
#To get a new column
df["round"] = df[['stageClassification']].apply(findStage,axis=1)
df = df.drop(['stageClassification'], axis=1)
df = df.reset_index()
df = df.drop(['startupID'], axis=1)


In [150]:
#Convert business Model to 0 or 1
#0 => B2B 1 => B2C
def findTypeOfBusiness(classification):
    typeOfBusiness, names = classification
    if(type(typeOfBusiness)==float):
        return -1
    elif(typeOfBusiness.find("B2B")!=-1):
        return 0
    elif(typeOfBusiness.find("B2C")!=-1):
        return 1


df["businessModelInt"] = df[['businessModel','keyword']].apply(findTypeOfBusiness,axis=1)
df = df.drop(['businessModel'], axis=1)



In [151]:
df.ICBindustry.unique()
#Technology is 0, Consumer Services 1, Consumer Goods is 2, Financials is 3,
#Industrials is 4,  Healthcare is 5 and blank is -1
def findIndustry(classification):
    round_name, new = classification
    if(type(round_name)==float):
        return -1
    if(round_name.find("Technology")!=-1):
        return 0
    elif (round_name.find("Consumer Services")!=-1):
        return 1
    elif(round_name.find("Consumer Goods")!=-1):
        return 2
    elif(round_name.find("Financials")!=-1):
        return 3
    elif(round_name.find("Industrials")!=-1):
        return 4
    elif(round_name.find("Healthcare")!=-1):
        return 5
    else:
        return -1
#To get a new column
df["industry"] = df[['ICBindustry','keyword']].apply(findIndustry,axis=1)
df = df.drop(['ICBindustry'], axis=1)


In [152]:
df.startupClassification.unique()
#1 if internet is present in the core business, 0 is not
def isInternet(classification):
    round_name, new = classification
    if(type(round_name)==float):
        return 0
    if(round_name.find("Internet")!=-1):
        return 1
    elif (round_name.find("Consumer Services")!=-1):
        return 0
#To get a new column
df["internet"] = df[['startupClassification','keyword']].apply(isInternet,axis=1)
df = df.drop(['startupClassification'], axis=1)


In [154]:
df.startupStatus.unique()
#1 if it is an angel investment, 0 if it is not
def setNature(classification):
    round_name, new = classification
    if(type(round_name)==float):
        return 1
    elif(round_name.find("VC Funded")!=-1):
        return 0
    elif (round_name.find("Angel investment")!=-1):
        return 1
    else: 
        return 1
#To get a new column
df["natureOfInvestment"] = df[['startupStatus','keyword']].apply(setNature,axis=1)
df = df.drop(['internet','keyword', 'investorType', 'investmentStage','startupName','groupClassification','startupStatus'], axis=1)

#do angel or vc that is necessart
#df = df.drop(['keyword', 'investorType', 'investmentStage','startupName','groupClassification'])

In [157]:
df.to_csv('datasets/modified.csv')
